In [1]:
import json

In [2]:
small = "stexpanded"
big = "memoryalpha"

In [3]:
mappings_file_small = "./_input/mappings/" + small + ".json"
mappings_file_big = "./_input/mappings/" + big + ".json"

gold_pairs_file = "./_input/gold_pairs/" + small + "-" + big + ".txt"
exact_match_file = "./_input/exact_match/" + small + "-" + big + ".json"
found_pairs_file = "./_input/found_pairs/" + small + "-" + big + ".txt"

In [4]:
with open(mappings_file_small) as file:
    mappings_small = {str(v): k for k, v in json.load(file).items()}
    mappings_small_reversed = {v: k for k, v in mappings_small.items()}

with open(mappings_file_big) as file:
    mappings_big = {str(v): k for k, v in json.load(file).items()}
    mappings_big_reversed = {v: k for k, v in mappings_big.items()}

with open(gold_pairs_file) as gpf:
    gold_pairs = []
    for line in gpf:
        numbers_list = [int(num) for num in line.strip().split(";")]
        gold_pairs.append(numbers_list)

with open(found_pairs_file) as fpf:
    found_pairs = []
    found_pairs_score = []
    for line in fpf:
        num_1 = int(mappings_small_reversed[line.strip().split("###")[0]])
        num_2 = int(mappings_big_reversed[line.strip().split("###")[1]])
        num_3 = float(line.strip().split("###")[2])
        numbers_list = [num_1, num_2]
        found_pairs.append(numbers_list)
        numbers_list_score = [num_1, num_2, num_3]
        found_pairs_score.append(numbers_list_score)

gold_pairs_left = {pair[0] for pair in gold_pairs}
gold_pairs_right = {pair[1] for pair in gold_pairs}

with open(exact_match_file) as file:
    exact_match = json.load(file)

exact_match_left = {pair[0] for pair in exact_match}

In [5]:
def compute_precision_recall_f1(gold_input, found_input):

    gold_set = set(tuple(pair) for pair in gold_input)
    found_set = set(tuple(pair) for pair in found_input)

    true_positives = gold_set & found_set
    tp = len(true_positives)
    fn = len(gold_set - found_set)
    fp = len(found_set - gold_set)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return precision, recall, f1

In [6]:
def print_found_but_not_gold(gold_input, found_input):

    found_but_not_gold = [[mappings_small[str(pair[0])], mappings_big[str(pair[1])]]
                          for pair in found_input if pair not in gold_input]

    for item in found_but_not_gold:
        print(item[0])
        print(item[1])
        print()

    print(len(found_but_not_gold))

In [7]:
def print_gold_not_found(gold_input, found_input):

    gold_not_found = [[mappings_small[str(pair[0])], mappings_big[str(pair[1])]]
                          for pair in gold_input if pair not in found_input]

    for item in gold_not_found:
        print(item[0])
        print(item[1])
        print()

    print(len(gold_not_found))

In [8]:
def clean_found_pairs(found_input_score, threshold):

    temp_dict = dict()

    for pair in found_input_score:
        if str(pair[1]) not in temp_dict:
            temp_dict[str(pair[1])] = pair[2]
        elif pair[2] > temp_dict[str(pair[1])]:
            temp_dict[str(pair[1])] = pair[2]

    output = []

    for pair in found_input_score:
        if ((pair[0] in gold_pairs_left or pair[1] in gold_pairs_right)
                and pair[2] > threshold
                and pair[2] == temp_dict[str(pair[1])]):
            output.append([pair[0], pair[1]])

    return output

In [9]:
def add_exact_match(found_input):

    output = []

    for em in exact_match:
        if em[0] in gold_pairs_left or em[1] in gold_pairs_right:
            output.append(em)

    for pair in found_input:
        if (pair[0] in gold_pairs_left or pair[1] in gold_pairs_right) and pair[0] not in exact_match_left:
            output.append(pair)

    return output

In [10]:
def calculate_threshold(found_input_score):

    temp_dict = dict()

    for pair in found_input_score:
        if str(pair[1]) not in temp_dict:
            temp_dict[str(pair[1])] = pair[2]
        elif pair[2] > temp_dict[str(pair[1])]:
            temp_dict[str(pair[1])] = pair[2]

    count = 0
    sum_score = 0.0
    for em in exact_match:
        if str(em[1]) in temp_dict:
            count += 1
            sum_score += temp_dict[str(em[1])]

    return sum_score / count

In [11]:
len(found_pairs)

15514

In [12]:
precision, recall, f1 = compute_precision_recall_f1(gold_pairs, found_pairs)

print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"F1 Score: {f1:.5f}")

Precision: 0.08212
Recall: 0.71613
F1 Score: 0.14734


In [13]:
reranker_threshold = calculate_threshold(found_pairs_score)
print(f"Threshold: {reranker_threshold:.5f}")

Threshold: 3.73778


In [14]:
found_pairs_cleaned = clean_found_pairs(found_pairs_score, reranker_threshold)

In [15]:
len(found_pairs_cleaned)

791

In [16]:
precision, recall, f1 = compute_precision_recall_f1(gold_pairs, found_pairs_cleaned)

print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"F1 Score: {f1:.5f}")

Precision: 0.88748
Recall: 0.39460
F1 Score: 0.54630


In [17]:
found_pairs_cleaned_em = add_exact_match(found_pairs_cleaned)

In [18]:
len(found_pairs_cleaned_em)

1761

In [19]:
precision, recall, f1 = compute_precision_recall_f1(gold_pairs, found_pairs_cleaned_em)

print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"F1 Score: {f1:.5f}")

Precision: 0.94151
Recall: 0.93198
F1 Score: 0.93672


In [20]:
print_found_but_not_gold(gold_pairs, found_pairs_cleaned_em)

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Torvin
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Torvin

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Fan_fiction
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Fan_fiction

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Cell
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Cell

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Enterprise_(NCC-1701)_(alternate_reality)
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/USS_Enterprise

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Planet_classifications
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Class_S

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Essex
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/USS_Essex

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Tia

In [21]:
print_gold_not_found(gold_pairs, found_pairs_cleaned_em)

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Alyson_Foxton
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/A.F.

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Aldebaran_Prime
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Aldebaran_III

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Argo_type
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Argo_(shuttlecraft)

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Armstrong_(NCC-57537)
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Armstrong_(disambiguation)

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Barstow
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Barstow_(Commodore)

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/SS_Bonaventure
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Bonaventure

http://dbkwik.webdatacommons.org/stexpa

In [24]:
for item in found_pairs_cleaned:
        print(mappings_small[str(item[0])])
        print(mappings_big[str(item[1])])
        print()

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/South_Carolina
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/South_Carolina

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Corporal
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Corporal

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Salok
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Sonak

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Aldebaran_whiskey
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Aldebaran_whiskey

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/IKS_Var%27tang
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/IKS_Vor%27nak

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Elephant
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Elephant

http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Hovercraft
http://db